# Fantasanremo

Prima di eseguire il codice importare il file ***FS.xlsx***

Pannello a sx -> File -> 'Carica in spazio di archiviazione della sessione'

In [1]:
!pip install ortools

In [2]:
# Import required libraries
from ortools.linear_solver import pywraplp
import pandas as pd

In [3]:
# Read data
data = pd.read_excel(r'/content/FS.xlsx')

In [4]:
# Clean data
data = data[['Artista', 'Punteggi FS', 'Costi']]
data = data.rename(columns={'Punteggi FS': 'Score'})
data = data[data['Artista'].notnull()]
data['Artista'] = data['Artista'].apply(lambda x: x.replace('"', ''))

data.head()

,Artista,Score,Costi
1,Achille Lauro - Domenica,50.0,32.0
2,Aka 7even - Perfetta così,35.0,23.0
3,Ana Mena - Duecentomila ore,25.0,17.0
4,Dargen D'Amico - Dove si balla,15.0,15.0
5,Ditonellapiaga con Rettore - Chimica,15.0,15.0


In [5]:
# Create the MIP Solver with the SCIP backend
solver = pywraplp.Solver.CreateSolver('SCIP')

In [6]:
# Define decision variables
for row in data.itertuples():
  exec(f'c{row.Index} = solver.IntVar(0.0, 1, "{row.Artista}")')

print('Number of variables =', solver.NumVariables())

Number of variables = 25


In [7]:
# Budget Constraint
# a*c1 + b*c2 + ... + n*cn <= 100
budget = solver.Constraint(-solver.infinity(), 100)

for row in data.itertuples():
  exec(f'budget.SetCoefficient(c{row.Index}, {row.Costi})')

In [8]:
# Team members Constraint
# c1 + c2 + c3 + c4 + c5 == 5
team = solver.Constraint(-solver.infinity(), 5)

for row in data.itertuples():
  exec(f'team.SetCoefficient(c{row.Index}, 1)')

In [9]:
# Objective function
objective = solver.Objective()

for row in data.itertuples():
  exec(f'objective.SetCoefficient(c{row.Index}, {row.Score})')

objective.SetMaximization()

In [10]:
# Solve
status = solver.Solve()

In [11]:
if status == pywraplp.Solver.OPTIMAL:
  print('Solution:')
  print('Objective value =', solver.Objective().Value())
  for var in solver.variables():
    if var.solution_value() == 1:
      print(f'Team member: {var.name()}')
else: 
  print('The problem does not have an optimal solution.')

Solution:
Objective value = 140.0
Team member: Achille Lauro - Domenica
Team member: Aka 7even - Perfetta così
Team member: Dargen D'Amico - Dove si balla
Team member: Ditonellapiaga con Rettore - Chimica
Team member: Yuman - “Ora e qui”
